# 抖音用户数据集

## 特征工程实践示例

In [ ]:
import pandas as pd
import numpy as np

# 加载数据
file_name = "user_personalized_features.csv"
df = pd.read_csv(file_name)

### 数据信息查看

In [ ]:
# Initial Data Inspection
print("Original Data Preview:")
print(df.head())
print("\nData Information:")
df.info()

### 删除重复的索引字段
删除原始数据中冗余的索引列Unnamed: 0.1和Unamed: 0。

In [ ]:
# Drop redundant index columns
df = df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])

print("DataFrame after dropping redundant columns:")
print(df.head())

### 数值特征工程

#### 数值离散化/分箱 (Binning)
年龄分箱 (Age_Group): 将Age特征离散化为 <20、20-35、36-50、51-65、65+ 五个年龄段。

In [ ]:
# Practice Case: Discretize Age into age groups.
bins = [0, 20, 35, 50, 65, np.inf]
labels = ['<20', '20-35', '36-50', '51-65', '65+']
df['Age_Group'] = pd.cut(df['Age'], bins=bins, labels=labels, right=True)
print("\nAge_Group (Age Binning):\n", df[['Age', 'Age_Group']].head())

#### 数值标准化/归一化 (Scaling)
- 收入Min-Max归一化 (Income_MinMaxScaled): 将Income收入特征归一化到0-1范围，适用于需要统一数值尺度的模型。
- 网站停留时间Z-score标准化 (Time_Spent_on_Site_Minutes_ZScaled): 对Time_Spent_on_Site_Minutes进行Z-score标准化，使其均值为0，标准差为1。

In [ ]:
# Practice Case: Standardize Income and Time_Spent_on_Site_Minutes.
# Min-Max Scaling for Income to [0, 1]
min_income = df['Income'].min()
max_income = df['Income'].max()
df['Income_MinMaxScaled'] = (df['Income'] - min_income) / (max_income - min_income)
print("\nIncome_MinMaxScaled:\n", df[['Income', 'Income_MinMaxScaled']].head())

# Z-score Standardization for Time_Spent_on_Site_Minutes
mean_time_spent = df['Time_Spent_on_Site_Minutes'].mean()
std_time_spent = df['Time_Spent_on_Site_Minutes'].std()
df['Time_Spent_on_Site_Minutes_ZScaled'] = (df['Time_Spent_on_Site_Minutes'] - mean_time_spent) / std_time_spent
print("\nTime_Spent_on_Site_Minutes_ZScaled:\n", df[['Time_Spent_on_Site_Minutes', 'Time_Spent_on_Site_Minutes_ZScaled']].head())

#### 特征交叉/组合 (Feature Interaction/Combination)
- 创建了 Income_Per_Minute_Spent，表示每分钟网站停留时间所对应的收入。
- 创建了 Purchase_Value_Per_Product，表示每浏览页面对应的总消费。

In [ ]:
# Practice Case: Create new features like Income_Per_Minute_Spent and Purchase_Value_Per_Product.
df['Income_Per_Minute_Spent'] = df['Income'] / (df['Time_Spent_on_Site_Minutes'] + 1e-6) # Add small epsilon to avoid division by zero
print("\nIncome_Per_Minute_Spent:\n", df[['Income', 'Time_Spent_on_Site_Minutes', 'Income_Per_Minute_Spent']].head())

df['Purchase_Value_Per_Product'] = df['Total_Spending'] / (df['Pages_Viewed'] + 1e-6) # Assuming Pages_Viewed is a proxy for "product interaction" or just another numeric column to combine
print("\nPurchase_Value_Per_Product:\n", df[['Total_Spending', 'Pages_Viewed', 'Purchase_Value_Per_Product']].head())

#### 多项式特征 (Polynomial Features)

In [ ]:
# Practice Case: Create a squared term for Age.
df['Age_Squared'] = df['Age'] ** 2
print("\nAge_Squared:\n", df[['Age', 'Age_Squared']].head())

### 日期时间特征工程
- 最后登录日期 (Last_Login_Date): 基于当前的日期 (2025-07-08) 和 Last_Login_Days_Ago 计算出精确的最后登录日期。
- 日期时间组件提取: 从 Last_Login_Date 中提取了 Last_Login_Year (年)、Last_Login_Month (月)、Last_Login_Day (日) 和 Last_Login_DayOfWeek (星期几)。

In [ ]:
print("\n--- 5. Date/Time Feature Engineering ---")

# The current date is Tuesday, July 8, 2025.
current_date = pd.to_datetime('2025-07-08')

# Calculate Last_Login_Date
df['Last_Login_Date'] = current_date - pd.to_timedelta(df['Last_Login_Days_Ago'], unit='D')
print("\nLast_Login_Date (derived from Last_Login_Days_Ago):\n", df[['Last_Login_Days_Ago', 'Last_Login_Date']].head())

# Extract year, month, day, day of week from Last_Login_Date
df['Last_Login_Year'] = df['Last_Login_Date'].dt.year
df['Last_Login_Month'] = df['Last_Login_Date'].dt.month
df['Last_Login_Day'] = df['Last_Login_Date'].dt.day
df['Last_Login_DayOfWeek'] = df['Last_Login_Date'].dt.dayofweek # Monday=0, Sunday=6
print("\nExtracted Date/Time features:\n", df[['Last_Login_Date', 'Last_Login_Year', 'Last_Login_Month', 'Last_Login_Day', 'Last_Login_DayOfWeek']].head())

### 布尔型值特征工程
- 订阅状态编码 (Newsletter_Subscription_Encoded): 将布尔型的 Newsletter_Subscription 转换为数值型 (True 转换为 1，False 转换为 0)。

In [ ]:
print("\n--- 6. Boolean Feature Engineering ---")

# Convert Newsletter_Subscription from boolean to integer (0 or 1)
df['Newsletter_Subscription_Encoded'] = df['Newsletter_Subscription'].astype(int)
print("\nNewsletter_Subscription_Encoded:\n", df[['Newsletter_Subscription', 'Newsletter_Subscription_Encoded']].head())

### 基于分组的特征工程
- 地点平均总消费 (Location_Avg_Total_Spending): 计算了每个 Location (地点) 的用户平均 Total_Spending (总消费)。
- 产品类别平均订单价值标准差 (Product_Category_Preference_Std_Avg_Order_Value): 计算了每个 Product_Category_Preference (产品类别偏好) 的 Average_Order_Value (平均订单价值) 的标准差。

In [ ]:
print("\n--- 7. Group-based Feature Engineering ---")

# Calculate average Total_Spending per Location
df['Location_Avg_Total_Spending'] = df.groupby('Location')['Total_Spending'].transform('mean')
print("\nLocation_Avg_Total_Spending:\n", df[['Location', 'Total_Spending', 'Location_Avg_Total_Spending']].head())

# Calculate standard deviation of Average_Order_Value per Product_Category_Preference
df['Product_Category_Preference_Std_Avg_Order_Value'] = df.groupby('Product_Category_Preference')['Average_Order_Value'].transform('std')
print("\nProduct_Category_Preference_Std_Avg_Order_Value:\n", df[['Product_Category_Preference', 'Average_Order_Value', 'Product_Category_Preference_Std_Avg_Order_Value']].head())

### 类别特征程

#### 独热编码(One-Hot Encoding)
- 独热编码: 对Gender(性别) 和Location (地点) 进行了独热编码，将它们转换为多个二进制列。

In [ ]:
# Apply to Gender and Location.
df = pd.get_dummies(df, columns=['Gender', 'Location'], prefix=['Gender', 'Location'])
print("\nDataFrame after One-Hot Encoding (Gender, Location):\n", df.head())

#### 频率编码(Frequency Encoding)
- 对Product_Category_Preference进行了频率编码，用每个产品类别的出现频率替换其原始类别值。
- 对Interests (兴趣) 进行了频率编码，用每个兴趣类别的出现频率替换其原始类别值。

In [ ]:
# Apply to Product_Category_Preference and Interests.
product_category_frequency = df['Product_Category_Preference'].value_counts(normalize=True)
df['Product_Category_Preference_FreqEncoded'] = df['Product_Category_Preference'].map(product_category_frequency)
print("\nProduct_Category_Preference_FreqEncoded:\n", df[['Product_Category_Preference', 'Product_Category_Preference_FreqEncoded']].head())

interests_frequency = df['Interests'].value_counts(normalize=True)
df['Interests_FreqEncoded'] = df['Interests'].map(interests_frequency)
print("\nInterests_FreqEncoded:\n", df[['Interests', 'Interests_FreqEncoded']].head())

### 处理缺失值

In [ ]:
print("\n--- 8. Handling Missing Values (Check only) ---")
print("\nMissing values before any imputation:\n", df.isnull().sum())
# 由info()的输出可知，该数据集没有缺失值，因此无须进行任何的缺失值处理操作。若存在缺失值的情况，可以使用类似如下的方法来进行缺失值填充。
# df['Column'].fillna(df['Column'].mean(), inplace=True) 

### 处理异常值
- 总消费封顶 (Total_Spending_Capped): 使用 IQR (四分位距) 方法检测并对 Total_Spending (总消费) 中的异常值进行了封顶处理，将超出上下限的值替换为相应的限制值。

In [ ]:
print("\n--- 9. Handling Outliers (Capping) ---")

# Practice Case: Outlier capping on Total_Spending using IQR.
Q1_spending = df['Total_Spending'].quantile(0.25)
Q3_spending = df['Total_Spending'].quantile(0.75)
IQR_spending = Q3_spending - Q1_spending
lower_bound_spending = Q1_spending - 1.5 * IQR_spending
upper_bound_spending = Q3_spending + 1.5 * IQR_spending

print(f"\nTotal_Spending IQR Detection: Q1={Q1_spending}, Q3={Q3_spending}, IQR={IQR_spending}")
print(f"Lower Bound (Total_Spending): {lower_bound_spending}")
print(f"Upper Bound (Total_Spending): {upper_bound_spending}")

# Apply capping
df['Total_Spending_Capped'] = np.where(
    df['Total_Spending'] < lower_bound_spending,
    lower_bound_spending,
    np.where(df['Total_Spending'] > upper_bound_spending, upper_bound_spending, df['Total_Spending'])
)
print("\nTotal_Spending_Capped (first 10 rows to show potential changes):\n", df[['Total_Spending', 'Total_Spending_Capped']].head(10))

# Final check of the DataFrame with new features
print("\nFinal DataFrame with engineered features preview:")
print(df.head())
print("\nFinal DataFrame Info:")
df.info()

### 保存处理后的结果

In [ ]:
# Save the transformed data to a CSV file
df.to_csv('engineered_user_features.csv', index=False)

## 绘图示例

### 环境准备与数据加载

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 加载上一节生成的特征工程后的数据
df = pd.read_csv('engineered_user_features.csv')

# 设置 Matplotlib 风格，让图表更美观
plt.style.use('seaborn-v0_8-darkgrid')
# 设置中文字体（根据你的系统选择合适的字体，例如 'SimHei' 或 'Microsoft YaHei'）
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS'] # For macOS
# plt.rcParams['font.sans-serif'] = ['SimHei'] # For Windows/Linux
plt.rcParams['axes.unicode_minus'] = False # 解决负号显示问题

print("特征工程后数据预览:")
print(df.head())
print("\n数据信息:")
df.info()

### 单变量分布分析

#### 直方图 (Histograms) - 数值特征分布
- 直方图显示了数值数据在不同区间内的频率分布。
- 示例：绘制Age和Income的直方图。

In [ ]:
print("\n--- 2.1 直方图 (Histograms) ---")

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1) # 1行2列，第1个图
sns.histplot(df['Age'], bins=10, kde=True, color='skyblue')
plt.title('用户年龄分布')
plt.xlabel('年龄')
plt.ylabel('数量')

plt.subplot(1, 2, 2) # 1行2列，第2个图
sns.histplot(df['Income'], bins=15, kde=True, color='lightcoral')
plt.title('用户收入分布')
plt.xlabel('收入')
plt.ylabel('数量')

plt.tight_layout()
plt.show()

#### 条形图 (Bar Plots) - 类别特征分布
- 条形图用于显示类别特征中每个类别的计数或频率。
- 实践案例：绘制 Gender 和 Age_Group 的分布。

In [ ]:
print("\n--- 2.2 条形图 (Bar Plots) ---")

plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.countplot(data=df, x='Gender_Male', palette='viridis') # Using one-hot encoded Gender
plt.xticks([0, 1], ['Female', 'Male']) # Manually set labels based on Gender_Male
plt.title('用户性别分布')
plt.xlabel('性别')
plt.ylabel('数量')

plt.subplot(1, 2, 2)
sns.countplot(data=df, x='Age_Group', order=df['Age_Group'].value_counts().index, palette='magma')
plt.title('用户年龄组分布')
plt.xlabel('年龄组')
plt.ylabel('数量')

plt.tight_layout()
plt.show()

#### 箱线图 (Box Plots) - 数值特征的统计概览和异常值检测
- 箱线图显示了数据的中位数、四分位数、离群点等，非常适合检测异常值。
- 实践案例： 绘制 Total_Spending 的箱线图，观察异常值处理效果。

In [ ]:
print("\n--- 2.3 箱线图 (Box Plots) ---")

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
sns.boxplot(y=df['Total_Spending'], color='lightgreen')
plt.title('总消费分布 (原始)')
plt.ylabel('总消费')

plt.subplot(1, 2, 2)
sns.boxplot(y=df['Total_Spending_Capped'], color='lightpink')
plt.title('总消费分布 (封顶后)')
plt.ylabel('总消费 (封顶)')

plt.tight_layout()
plt.show()

### 双变量关系分析
- 探索两个特征之间的关系是发现有用模式的关键。

#### 散点图 (Scatter Plots) - 数值特征之间关系
- 散点图用于显示两个数值变量之间的关系。
- 实践案例： 绘制 Income 和 Total_Spending 之间的关系，以及 Time_Spent_on_Site_Minutes 和 Pages_Viewed 之间的关系。

In [ ]:
print("\n--- 3.1 散点图 (Scatter Plots) ---")

plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.scatterplot(data=df, x='Income', y='Total_Spending', hue='Gender_Male', palette='coolwarm', alpha=0.7)
plt.title('收入 vs. 总消费')
plt.xlabel('收入')
plt.ylabel('总消费')
plt.legend(title='性别 (1=男, 0=女)')

plt.subplot(1, 2, 2)
sns.scatterplot(data=df, x='Time_Spent_on_Site_Minutes', y='Pages_Viewed', color='purple', alpha=0.6)
plt.title('网站停留时间 vs. 浏览页面数')
plt.xlabel('网站停留时间 (分钟)')
plt.ylabel('浏览页面数')

plt.tight_layout()
plt.show()

#### 小提琴图 (Violin Plots) - 类别与数值特征关系
- 小提琴图结合了箱线图和核密度估计图的优点，显示了数值特征在不同类别下的分布。
- 实践案例： 绘制 Location 对 Income 的影响。

In [ ]:
print("\n--- 3.2 小提琴图 (Violin Plots) ---")

# 重建一个临时的 'Location' 列，以便用其原始类别值进行绘图
# 或者，直接使用 one-hot 编码的列进行分类对比
# 为了展示清晰，这里假设我们能直接访问原始的 Location 列进行分组。
# 由于原始数据中 Location 有 'Rural', 'Suburban', 'Urban'，我们使用这些。
# 注意：在实际特征工程后，原始列可能已被替换。这里为了绘图方便，我们直接基于数值列和类别分组。
# 如果原始 Location 列已被删除，则需要根据 one-hot encoded 列重新创建分类标签。

# 从原始 df 中获取 Location 用于绘图 (或重新加载)
df_plot_original_location = pd.read_csv('user_personalized_features.csv').drop(columns=['Unnamed: 0.1', 'Unnamed: 0'])

plt.figure(figsize=(10, 6))
sns.violinplot(data=df_plot_original_location, x='Location', y='Income', palette='Set2')
plt.title('不同地点用户的收入分布')
plt.xlabel('地点')
plt.ylabel('收入')
plt.show()

#### 分组柱状图 (Grouped Bar Charts) - 类别与类别特征关系
- 用于比较不同类别组的计数或频率。
- 实践案例： 绘制不同 Age_Group 中 Gender 的分布。

In [ ]:
print("\n--- 3.3 分组柱状图 (Grouped Bar Charts) ---")

# 使用melt函数将Gender_Female和Gender_Male转换成一个统一的性别列，方便绘图
df_melted_gender_age = df.melt(id_vars=['Age_Group'], value_vars=['Gender_Female', 'Gender_Male'],
                               var_name='Gender_Encoded', value_name='Count')
# 筛选出 Count 为 1 的行，因为独热编码只有0和1
df_melted_gender_age = df_melted_gender_age[df_melted_gender_age['Count'] == 1]
df_melted_gender_age['Gender'] = df_melted_gender_age['Gender_Encoded'].apply(lambda x: 'Female' if 'Female' in x else 'Male')

plt.figure(figsize=(12, 7))
sns.countplot(data=df_melted_gender_age, x='Age_Group', hue='Gender', palette='pastel',
              order=sorted(df['Age_Group'].dropna().unique().astype(str))) # Ensure correct order
plt.title('不同年龄组的用户性别分布')
plt.xlabel('年龄组')
plt.ylabel('数量')
plt.legend(title='性别')
plt.show()

### 多变量关系分析
- 探索三个或更多特征之间的关系，通常需要更复杂的图表或组合图。

#### 相关性热力图 (Correlation Heatmaps)
- 相关性热力图显示了数据集中所有数值特征之间的相关性。这是发现高度相关（可能冗余）或与目标变量强相关特征的重要工具。
- 实践案例： 绘制所有数值特征的相关性热力图。

In [ ]:
print("\n--- 4.1 相关性热力图 (Correlation Heatmaps) ---")

# 选择数值特征
numeric_cols = df.select_dtypes(include=np.number).columns
# 排除 User_ID 和可能不需要分析相关性的编码特征（如 Last_Login_Year/Month/Day）
# 但为了全面展示，我们先包含大部分，再考虑排除
features_for_correlation = [col for col in numeric_cols if col not in ['User_ID', 'Newsletter_Subscription_Encoded',
                                                                        'Last_Login_Year', 'Last_Login_Month', 'Last_Login_Day', 'Last_Login_DayOfWeek',
                                                                        'Gender_Female', 'Gender_Male', 'Location_Rural', 'Location_Suburban', 'Location_Urban']]
correlation_matrix = df[features_for_correlation].corr()

plt.figure(figsize=(14, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('数值特征相关性热力图')
plt.show()

#### Pair Plot (散点图矩阵)
- sns.pairplot 可以绘制数据集中所有数值特征两两之间的散点图，并在对角线上绘制每个特征的分布图。
- 实践案例： 对一些核心数值特征进行 Pair Plot 分析。

In [ ]:
print("\n--- 4.2 Pair Plot (散点图矩阵) ---")

# 选择一些核心数值特征
selected_features_for_pairplot = ['Age', 'Income', 'Total_Spending', 'Purchase_Frequency', 'Time_Spent_on_Site_Minutes']

# 可以根据Gender进行着色
sns.pairplot(df, vars=selected_features_for_pairplot, hue='Gender_Male', palette='viridis', diag_kind='kde')
plt.suptitle('主要数值特征的散点图矩阵 (按性别着色)', y=1.02) # y调整标题位置
plt.show()